## Script to create zonal mean of polygons (in shp file)

In [1]:
import os
from time import time
import xarray as xr
import rioxarray
import fiona
import rasterio
import rasterio.mask as msk
import numpy as np
from datetime import date, datetime
from dateutil.rrule import rrule, DAILY
import pandas as pd

#### Function to perform zonal mean on polygon and output data in csv file

In [2]:
def zonal_stats(shape_path, infile, parameter,
                dict={"Year": [], "DOY": [], "parameter": [], "YYYYDOY": [], 'ID': [], 'name': [], 'mean': []}):
    # === Zonal Stats ===
    with fiona.open(shape_path, 'r') as shp:
        features = [feature for feature in shp]
        with rasterio.open(infile) as src:
            for f in features:
                src_shp = [f['geometry']]
                outimage, out_transform = msk.mask(src, src_shp, crop=True)
                #print(outimage)
                #print('type: ', type(outimage))
                ws_id = f['id']
                ws_name = f['properties']['Name']
                ws_ar = outimage
                ws_ar[ws_ar > 1000.0] = np.nan
                ws_mean = np.nanmean(ws_ar)
                #print(ws_mean)
                ws_year = infile.split('.')[0][-7:-3]
                ws_doy = infile.split('.')[0][-3:]
                dict['name'].append(ws_name)
                dict['mean'].append(ws_mean)
                dict['Year'].append(ws_year)
                dict['DOY'].append(ws_doy)
                dict['YYYYDOY'].append('{}_{}'.format(ws_year,ws_doy))
                dict['parameter'].append(parameter)
                dict['ID'].append(ws_id)
    return dict

In [3]:
path_mos_dat = 's3://dev-et-data/in/USA_data/precip_gridmet_tiffs' # /year
#shape_path = 'Small Watersheds 128_Aggregated.shp'
shape_path = 'Small_Watersheds_128_Aggregated_37.shp'
csv_output = '.'  #"." represent the current dir in Linux or type '/home/jupyter-kagone/postprocess'
parameter = 'prec'

a = datetime(2000, 1, 1)
#b = datetime(2000, 1, 3)
b = datetime(2019, 12, 31)

for dt in rrule(DAILY, dtstart=a, until=b):
    #print(dt.strftime("%Y-%m-%d")) # this returns string
    year = dt.strftime("%Y")
    month = dt.strftime("%m")
    day = dt.strftime("%d")
    doy = dt.timetuple().tm_yday
    print('Day: {} = {}-{}-{}'.format(doy, year, month, day))
        
    in_file = os.path.join(path_mos_dat, year, '{}_{}{:03d}.tif'.format(parameter, year, doy))
    print(in_file)

    if dt == a:
        outdict = zonal_stats(shape_path, in_file, parameter)
    elif dt < b:
        outdict = zonal_stats(shape_path, in_file, parameter, dict=outdict)
    else:
        finaldict = zonal_stats(shape_path, in_file, parameter, dict=outdict)

#csvdf = pd.DataFrame(finaldict, columns=['Year', 'DOY', 'YYYYDOY', 'parameter', 'ID',  'name', 'mean'])
csvdf = pd.DataFrame(finaldict, columns=['YYYYDOY', 'ID',  'mean'])
print(csvdf)
#csvdf = csvdf.pivot(index=None, columns='ID', values='mean')
#csvdf = pd.pivot_table(csvdf, values='mean', index='YYYYDOY', columns='ID').reset_index()

csvdf[['YYYY','DOY']] = csvdf.YYYYDOY.apply(lambda x: pd.Series(str(x).split("_"))) 
print('these are the columns ', csvdf.columns)

#cols = csvdf.columns
#cols_list = cols.sort_values()
#print(cols_list)


# shp_list = list(range(53))
# listf = []
# listf.append('YYYYDOY')
# listf.append('YYYY')
# listf.append('DOY')
# for elem in shp_list:
#     listf.append('{}'.format(elem))
#print('this is the listssssss ', listf)
#csvdf = csvdf[listf]
#csvdf = csvdf[cols_list]
print(csvdf)

csvdf.to_csv(os.path.join(csv_output, '{}_zonalmean_2000_2019_37.csv'.format(parameter)))
print('done')

Day: 1 = 2000-01-01
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000001.tif


/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: Mean of empty slice
  app.launch_new_instance()


Day: 2 = 2000-01-02
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000002.tif
Day: 3 = 2000-01-03
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000003.tif
Day: 4 = 2000-01-04
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000004.tif
Day: 5 = 2000-01-05
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000005.tif
Day: 6 = 2000-01-06
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000006.tif
Day: 7 = 2000-01-07
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000007.tif
Day: 8 = 2000-01-08
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000008.tif
Day: 9 = 2000-01-09
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000009.tif
Day: 10 = 2000-01-10
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000010.tif
Day: 11 = 2000-01-11
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000011.tif
Day: 12 = 2000-01-12
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/pr

Day: 91 = 2000-03-31
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000091.tif
Day: 92 = 2000-04-01
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000092.tif
Day: 93 = 2000-04-02
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000093.tif
Day: 94 = 2000-04-03
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000094.tif
Day: 95 = 2000-04-04
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000095.tif
Day: 96 = 2000-04-05
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000096.tif
Day: 97 = 2000-04-06
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000097.tif
Day: 98 = 2000-04-07
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000098.tif
Day: 99 = 2000-04-08
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000099.tif
Day: 100 = 2000-04-09
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000100.tif
Day: 101 = 2000-04-10
s3://dev-et-data/in/USA_data/precip_gridmet_tif

Day: 179 = 2000-06-27
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000179.tif
Day: 180 = 2000-06-28
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000180.tif
Day: 181 = 2000-06-29
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000181.tif
Day: 182 = 2000-06-30
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000182.tif
Day: 183 = 2000-07-01
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000183.tif
Day: 184 = 2000-07-02
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000184.tif
Day: 185 = 2000-07-03
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000185.tif
Day: 186 = 2000-07-04
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000186.tif
Day: 187 = 2000-07-05
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000187.tif
Day: 188 = 2000-07-06
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2000/prec_2000188.tif
Day: 189 = 2000-07-07
s3://dev-et-data/in/USA_data/precip_gr

...

Day: 299 = 2018-10-26
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2018/prec_2018299.tif
Day: 300 = 2018-10-27
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2018/prec_2018300.tif
Day: 301 = 2018-10-28
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2018/prec_2018301.tif
Day: 302 = 2018-10-29
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2018/prec_2018302.tif
Day: 303 = 2018-10-30
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2018/prec_2018303.tif
Day: 304 = 2018-10-31
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2018/prec_2018304.tif
Day: 305 = 2018-11-01
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2018/prec_2018305.tif
Day: 306 = 2018-11-02
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2018/prec_2018306.tif
Day: 307 = 2018-11-03
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2018/prec_2018307.tif
Day: 308 = 2018-11-04
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2018/prec_2018308.tif
Day: 309 = 2018-11-05
s3://dev-et-data/in/USA_data/precip_gr

Day: 22 = 2019-01-22
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019022.tif
Day: 23 = 2019-01-23
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019023.tif
Day: 24 = 2019-01-24
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019024.tif
Day: 25 = 2019-01-25
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019025.tif
Day: 26 = 2019-01-26
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019026.tif
Day: 27 = 2019-01-27
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019027.tif
Day: 28 = 2019-01-28
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019028.tif
Day: 29 = 2019-01-29
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019029.tif
Day: 30 = 2019-01-30
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019030.tif
Day: 31 = 2019-01-31
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019031.tif
Day: 32 = 2019-02-01
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs

Day: 110 = 2019-04-20
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019110.tif
Day: 111 = 2019-04-21
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019111.tif
Day: 112 = 2019-04-22
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019112.tif
Day: 113 = 2019-04-23
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019113.tif
Day: 114 = 2019-04-24
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019114.tif
Day: 115 = 2019-04-25
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019115.tif
Day: 116 = 2019-04-26
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019116.tif
Day: 117 = 2019-04-27
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019117.tif
Day: 118 = 2019-04-28
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019118.tif
Day: 119 = 2019-04-29
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019119.tif
Day: 120 = 2019-04-30
s3://dev-et-data/in/USA_data/precip_gr

Day: 198 = 2019-07-17
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019198.tif
Day: 199 = 2019-07-18
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019199.tif
Day: 200 = 2019-07-19
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019200.tif
Day: 201 = 2019-07-20
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019201.tif
Day: 202 = 2019-07-21
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019202.tif
Day: 203 = 2019-07-22
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019203.tif
Day: 204 = 2019-07-23
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019204.tif
Day: 205 = 2019-07-24
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019205.tif
Day: 206 = 2019-07-25
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019206.tif
Day: 207 = 2019-07-26
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019207.tif
Day: 208 = 2019-07-27
s3://dev-et-data/in/USA_data/precip_gr

Day: 286 = 2019-10-13
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019286.tif
Day: 287 = 2019-10-14
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019287.tif
Day: 288 = 2019-10-15
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019288.tif
Day: 289 = 2019-10-16
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019289.tif
Day: 290 = 2019-10-17
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019290.tif
Day: 291 = 2019-10-18
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019291.tif
Day: 292 = 2019-10-19
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019292.tif
Day: 293 = 2019-10-20
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019293.tif
Day: 294 = 2019-10-21
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019294.tif
Day: 295 = 2019-10-22
s3://dev-et-data/in/USA_data/precip_gridmet_tiffs/2019/prec_2019295.tif
Day: 296 = 2019-10-23
s3://dev-et-data/in/USA_data/precip_gr